# Машинное обучение, ФКН ВШЭ

## Практическое задание 4. Классификация

### Общая информация
Дата выдачи: 16.11.2024

Мягкий дедлайн: 28.11.2024

Жесткий дедлайн: 02.12.2024

### О задании

В этом задании вы:
- ознакомитесь с тем, что происходит "внутри" метода опорных векторов и логистической регрессии
- познакомитесь с калибровкой вероятности
- изучите методы трансформации переменных и методы отбора признаков
- попробуете оценить экономический эффект модели

----

#### Самостоятельная оценка результатов

Для удобства проверки, исходя из набора решенных задач, посчитайте свою максимальную оценку.

**Оценка**:

### Оценивание и штрафы

Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 10 баллов.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.

### Формат сдачи

Для сдачи задания переименуйте получившийся файл *.ipynb в соответствии со следующим форматом: homework-practice-04-linclass-__Username__.ipynb, где Username — ваша фамилия и имя на латинице именно в таком порядке (например, homework-practice-04-linclass-__IvanovIvan__.ipynb).

# Часть 1. SVM, LR и калибровка вероятностей (2 балла + 0.5 бонус)

In [ ]:
import numpy as np
import pandas as pd
# import polars as pl
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
# pl.Config().set_tbl_rows(100)
# pl.Config().set_tbl_cols(100)

#### __Задание 1.1  Сравнение методов__ (0.5 балла)



Сгенерируем синтетические данные.

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# фиксируем random_state для воспроизводимости результатов
X, y = make_classification(
    n_samples=10000, n_features=10, n_informative=5, n_redundant=5, random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

__Случайный классификатор__

Для начала зададим самую простую модель, которая на каждом объекте выдаёт случайный ответ. По тестовой выборке вычислим AUC-ROC, AUC-PR.

In [ ]:
from sklearn.dummy import DummyClassifier
random_classifier = DummyClassifier(strategy='uniform', random_state=42).fit(X_train, y_train)
y_random = random_classifier.predict_proba(X_test)[:,1]
y_random

**Вопрос:** решаем задачу бинарной классификации, но y\_random содержит какие-то дробные числа, а не 0/1. Почему?



**Ответ**: предсказываем вероятности вместо бинарного таргета 0/1

*Ниже приведен **пример** работы* со встроенными функциями `sklearn` для отрисовки ROC и PR кривых, сохранения метрик. Пайплайн можно изменять как вам удобно.

In [ ]:
from sklearn.metrics import average_precision_score

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import PrecisionRecallDisplay

from sklearn.metrics import roc_auc_score
from sklearn.metrics import RocCurveDisplay

In [ ]:
def depict_pr_roc(y_true, y_pred, classifier_name='Some Classifier', ax=None):
    if ax is None:
        fig, ax = plt.subplots(1, 2, figsize=(11, 5))

    print(classifier_name, 'metrics')
    PrecisionRecallDisplay.from_predictions(y_true, y_pred, ax=ax[0], name=classifier_name)
    print('AUC-PR: %.4f' % average_precision_score(y_true, y_pred))
    ax[0].set_title("PRC")
    ax[0].set_ylim(0, 1.1)

    RocCurveDisplay.from_predictions(y_true, y_pred, ax=ax[1], name=classifier_name)
    print('AUC-ROC: %.4f' % roc_auc_score(y_true, y_pred))
    ax[1].set_title("ROC")
    ax[1].set_ylim(0, 1.1)

    plt.tight_layout()
    plt.legend()


depict_pr_roc(y_test, y_random, 'Random Classifier')

In [ ]:
# dataframe для сравнения
# методов классификации по метрикам
df_metrics = pd.DataFrame(
    columns=['auc_pr', 'roc_auc_score', 'reg_const']
)
precision, recall, _ = precision_recall_curve(y_test, y_random)
# добавление очередной строки с характеристиками метода
df_metrics.loc['Random Classifier'] = [
      average_precision_score(y_test, y_random),
      roc_auc_score(y_test, y_random),
      0,
]

# по аналогии результаты следующих экспериментов можно будет собрать в табличку
df_metrics

__Support Vector Machine (Linear Kernel)__

Обучите метод опорных векторов.

Подберите параметр регуляризации `C` с точки зрения AUC-PR (можете воспользоваться кросс-валидацией или отделить валидационную выборку от обучающей).


In [ ]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import RandomizedSearchCV


linear_svc = LinearSVC(C=1.0,
                       penalty="l2",
                       tol=0.0001,
                       random_state=42,
                       max_iter=1000)

params_linear_svc = {
    'C': np.logspace(-9, 4, 20),
    'tol': [1e-5, 1e-4, 1e-3, 1e-2],
}

random_search_svc = RandomizedSearchCV(
    estimator=linear_svc,
    param_distributions=params_linear_svc,
    cv=5,
    n_iter=60,
    scoring='average_precision',
    n_jobs=-1,
    random_state=42,
    verbose=1
)

random_search_svc.fit(X_train, y_train)

print(f'Best parameters: {random_search_svc.best_params_}')

# Best parameters: {'tol': 0.001, 'C': np.float64(6.158482110660267e-05)}

  На тестовой части:
  - постройте ROC и PR кривые,
  - посчитайте AUC-ROC, AUC-PR

In [ ]:
from sklearn.calibration import CalibratedClassifierCV

# LinearSVC doesn't output probabilities directly, so we calibrate it
calibrated_svc = CalibratedClassifierCV(random_search_svc.best_estimator_, cv=5)
calibrated_svc.fit(X_train, y_train)

# Get probability predictions for test set
y_svc_proba = calibrated_svc.predict_proba(X_test)[:, 1]

# Plot ROC and PR curves
depict_pr_roc(y_test, y_svc_proba, 'Linear SVC (Calibrated)')

# Update metrics dataframe
df_metrics.loc['Linear SVC'] = [
    average_precision_score(y_test, y_svc_proba),
    roc_auc_score(y_test, y_svc_proba),
    random_search_svc.best_params_['C'],
]


Проанализируйте, как себя ведут обе кривые:
- Что происходит при увеличении порога? Как бы вы это проинтерпретировали?
- Монотонные ли кривые? Как вы это объясните?

Ответ:

-ROC всегда монотонна т.к. TRP and FPR при уменьшении порога будут монотонно расти (знаменатель постоянный у обеих), но TP и FP всегда увеличивается 

-Критическое различие: знаменатель precision  меняется при изменении порога

__Logistic Regression__


Аналогичное задание для логистической регрессии с L2 регуляризатором:


*   подберите гиперпараметр C, используя метрику AUC-PR
*   нарисуйте ROC, PR кривые для тестовой части
*   выведите метрики для тестовых данных и сравните их с результатами случайного классификатора



In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(
                            C=1.0,
                            penalty="l2",
                            tol=0.0001,
                            random_state=42,
                            max_iter=1000
)

params_log_reg = {
    'C': np.logspace(-9, 4, 20),
    'tol': np.logspace(-8, -2, 10)
}

random_search_log_reg = RandomizedSearchCV(
    estimator=log_reg,
    param_distributions=params_log_reg,
    cv=5,
    n_iter=60,
    scoring='average_precision',
    n_jobs=-1,
    random_state=42,
    verbose=1
)

random_search_log_reg.fit(X_train, y_train)

print(f'Best parameters: {random_search_log_reg.best_params_}')

# Best parameters: {'tol': np.float64(2.154822e-07), 'C': np.float64(0.00029763)}

In [ ]:
# Get probability predictions for test set
log_reg.fit(X_train, y_train)

y_log_reg_proba = log_reg.predict_proba(X_test)[:, 1]

# Plot ROC and PR curves
depict_pr_roc(y_test, y_log_reg_proba, 'Logistic Regression + L2')

# Update metrics dataframe
df_metrics.loc['Logistic Regression + L2'] = [
    average_precision_score(y_test, y_log_reg_proba),
    roc_auc_score(y_test, y_log_reg_proba),
    random_search_log_reg.best_params_['C'],
]

Нарисуйте ROC, PR кривые для тестовой части для всех 3 классификаторов на одном графике

In [ ]:
fig, (ax_pr, ax_roc) = plt.subplots(1, 2, figsize=(14, 5))

# Random Classifier
print('Random Classifier metrics')
PrecisionRecallDisplay.from_predictions(y_test, y_random, ax=ax_pr, name='Random Classifier')
print('AUC-PR: %.4f' % average_precision_score(y_test, y_random))

RocCurveDisplay.from_predictions(y_test, y_random, ax=ax_roc, name='Random Classifier')
print('AUC-ROC: %.4f' % roc_auc_score(y_test, y_random))

# Linear SVC (Calibrated)
print('\nLinear SVC (Calibrated) metrics')
PrecisionRecallDisplay.from_predictions(y_test, y_svc_proba, ax=ax_pr, name='Linear SVC (Calibrated)')
print('AUC-PR: %.4f' % average_precision_score(y_test, y_svc_proba))

RocCurveDisplay.from_predictions(y_test, y_svc_proba, ax=ax_roc, name='Linear SVC (Calibrated)')
print('AUC-ROC: %.4f' % roc_auc_score(y_test, y_svc_proba))

# Logistic Regression
print('\nLogistic Regression + L2 metrics')
PrecisionRecallDisplay.from_predictions(y_test, y_log_reg_proba, ax=ax_pr, name='Logistic Regression + L2')
print('AUC-PR: %.4f' % average_precision_score(y_test, y_log_reg_proba))

RocCurveDisplay.from_predictions(y_test, y_log_reg_proba, ax=ax_roc, name='Logistic Regression + L2')
print('AUC-ROC: %.4f' % roc_auc_score(y_test, y_log_reg_proba))

ax_pr.set_title("Precision-Recall Curve")
ax_pr.set_ylim(0, 1.1)
ax_roc.set_title("ROC Curve")
ax_roc.set_ylim(0, 1.1)

plt.tight_layout()
plt.show()

**Вопрос:** Сравните результаты LR и SVM с точки зрения всех вычисленных критериев качества, объясните различия (если они есть).



**Ответ:** # отличий почти нет

#### __Задание 1.2. Визуализация в подходах SVM, LR__ (0.5 балла)



В названии метода опорных векторов присутствуют некоторые "опорные векторы". По сути, это объекты из обучающей выборки, которые задали положение разделяющей гиперплоскости.

* Сгенерируйте синтетические данные с помощью `make_classification` __с 2 признаками__, обучите на нём метод опорных векторов. Не забудьте зафиксировать seed для воспроизводимости

* Визуализируйте разделяющую прямую, все объекты и выделите опорные векторы. Ниже есть шаблоны, можете воспользоваться ими, либо написать своё

In [ ]:
X, y = make_classification(
    n_samples=10000, n_features=2, n_informative=2, n_redundant=0, random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

linear_svc = SVC(kernel='linear', C=1.0, tol=0.00001, random_state=42, max_iter=1000)

params_linear_svc = {
    'C': np.logspace(-9, 4, 20),
}

random_search_svc = RandomizedSearchCV(
    estimator=linear_svc,
    param_distributions=params_linear_svc,
    cv=5,
    n_iter=60,
    scoring='average_precision',
    n_jobs=-1,
    random_state=42,
    verbose=0
)

random_search_svc.fit(X_train, y_train)

print(f'Best parameters: {random_search_svc.best_params_}')

In [ ]:
best_linear_svc = SVC(kernel='linear', **random_search_svc.best_params_, random_state=42)

best_linear_svc.fit(X_train, y_train)

y_pred = best_linear_svc.predict(X_test)

df = pd.concat([pd.DataFrame(X_test, columns=['x1', 'x2']), pd.DataFrame(y_pred, columns=['y_pred'])], axis=1)

print(list(best_linear_svc.coef_[0]))
print(best_linear_svc.intercept_[0])

In [ ]:
sv = best_linear_svc.support_vectors_

In [ ]:
w1, w2 = best_linear_svc.coef_[0]
b = best_linear_svc.intercept_[0]

# Диапазон для оси x
xx = np.linspace(-5, 6, 100)
yy = -(w1 * xx + b) / w2

y_0 = df[df['y_pred'] == 0]
y_1 = df[df['y_pred'] == 1]

plt.figure(figsize=(14, 14))
plt.plot(xx, yy, color='green', linewidth=2, label='Гиперплоскость')

# Маржины (границы отступа): w1*x1 + w2*x2 + b = ±1
yy_margin_plus = -(w1 * xx + b + 1) / w2
yy_margin_minus = -(w1 * xx + b - 1) / w2

plt.plot(xx, yy_margin_plus, 'r--', linewidth=2, label='Верхний margin')
plt.plot(xx, yy_margin_minus, 'r--', linewidth=2, label='Нижний margin')

plt.scatter(y_0['x1'], y_0['x2'], c='red', label='Class 0', alpha=0.7)
plt.scatter(y_1['x1'], y_1['x2'], c='blue', label='Class 1', alpha=0.7)

# НАСТОЯЩИЕ support vectors из модели LinearSVC
# LinearSVC не предоставляет .support_vectors_ напрямую, поэтому извлекаем их
decision_vals = best_linear_svc.decision_function(df[['x1', 'x2']])
norm_w = np.linalg.norm([w1, w2])

# Точные опорные векторы: |decision_function(x)| / ||w|| <= 1 + epsilon
distances = np.abs(decision_vals) / norm_w
sv_mask_true = np.abs(decision_vals) <= 1 + 1e-4  # Точные SV (|w*x + b| <= 1)

# Разделяем по классам настоящие SV
df_sv_true = df[sv_mask_true].copy()
sv_class_0_true = df_sv_true[df_sv_true['y_pred'] == 0]
sv_class_1_true = df_sv_true[df_sv_true['y_pred'] == 1]

print(f"True SV Class 0: {len(sv_class_0_true)}")
print(f"True SV Class 1: {len(sv_class_1_true)}")

# ✅ НАСТОЯЩИЕ ОПОРНЫЕ ВЕКТОРЫ - БОЛЬШИЕ ЗВЁЗДЫ
plt.scatter(sv_class_0_true['x1'], sv_class_0_true['x2'], 
           c='gold', s=400, marker='+', 
           edgecolors='black', linewidth=3,
           label=f'TRUE SV Class 0 (n={len(sv_class_0_true)})', zorder=10)

plt.scatter(sv_class_1_true['x1'], sv_class_1_true['x2'], 
           c='limegreen', s=400, marker='+', 
           edgecolors='black', linewidth=3,
           label=f'TRUE SV Class 1 (n={len(sv_class_1_true)})', zorder=10)

# ПСЕВДО-опорные векторы (близкие к маржину) - меньшие звёздочки для сравнения
pseudo_sv_mask = (0.95 < distances) & (distances <= 1.1)
df_pseudo_sv = df[pseudo_sv_mask].copy()
pseudo_sv_0 = df_pseudo_sv[df_pseudo_sv['y_pred'] == 0]
pseudo_sv_1 = df_pseudo_sv[df_pseudo_sv['y_pred'] == 1]

plt.scatter(pseudo_sv_0['x1'], pseudo_sv_0['x2'], 
           c='darkred', s=150, marker='*', 
           edgecolors='white', linewidth=1.5,
           label=f'Pseudo-SV Class 0 (n={len(pseudo_sv_0)})', alpha=0.8)

plt.scatter(pseudo_sv_1['x1'], pseudo_sv_1['x2'], 
           c='darkblue', s=150, marker='*', 
           edgecolors='white', linewidth=1.5,
           label=f'Pseudo-SV Class 1 (n={len(pseudo_sv_1)})', alpha=0.8)

plt.xlabel('x1')
plt.ylabel('x2')

plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.axis('equal')
plt.tight_layout()
plt.show()

# Проверка support_vectors_ (если доступно)
print("best_linear_svc.support_vectors_:", best_linear_svc.support_vectors_ if hasattr(best_linear_svc, 'support_vectors_') else "Not available")
print("Количество SV по классам:", len(sv_class_0_true), len(sv_class_1_true))


In [ ]:
xx = np.linspace(X_train[:,0].min(), X_train[:,0].max(), 30)
yy = np.linspace(X_train[:,1].min(), X_train[:,1].max(), 30)
YY, XX = np.meshgrid(yy, xx)

In [ ]:
def plot_svm_2D(X, y, model,  plot_support=True):

    # создали сетку
    xx = np.linspace(X[:,0].min(), X[:,0].max(), 30)
    yy = np.linspace(X[:,1].min(), X[:,1].max(), 30)
    YY, XX = np.meshgrid(yy, xx)
    xy = np.vstack([XX.ravel(), YY.ravel()]).T

    # Ответы модели для сетки для отрисовки разделяющей прямой
    Z = # your code here

    plt.contour(XX, YY, Z, colors='k', levels=[-1, 0, 1], alpha=0.5, linestyles=['--', '-', '--'])

    # Отрисовали выборку
    plt.scatter(
        # your code here
    )

    # Отрисовали опорные векторы
    if plot_support:
        plt.scatter(
            # your code here
            label='support vectors',
            s=100,
            linewidth=1,
            edgecolor="blue",
            facecolors='none'
        )

    plt.legend()

plot_svm_2D(X, y, model)

**Вопрос:** какие объекты выделяются как "опорные"?



**Ответ:** # your answer here

В отличие от метода опорных векторов, логистическая регрессия не пытается построить разделяющую гиперплоскость с максимальным отступом, а приближает в каждой точке пространства объектов вероятность положительных ответов $p(y=+1|x)$. Попробуйте нарисовать это распределение на плоскости, не забудьте отметить на ней все объекты.

In [ ]:
def plot_logreg_2D(X, y, model):

    # создали сетку
    xx = np.linspace(X[:,0].min(), X[:,0].max(), 100)
    yy = np.linspace(X[:,1].min(), X[:,1].max(), 100)
    YY, XX = np.meshgrid(yy, xx)
    xy = np.vstack([XX.ravel(), YY.ravel()]).T

    # Ответы модели для сетки для отрисовки распределения
    Z = # your code here
    Z = Z.reshape((xx.shape[0], -1)).T

    image = plt.imshow(
        Z,
        interpolation='nearest',
        extent=(xx.min(), xx.max(), yy.min(), yy.max()),
        aspect='auto',
        origin='lower',
        cmap=plt.cm.PuOr_r
    )

    #Отрисовали выборку
    plt.scatter(
        # your code here
        cmap=plt.cm.Paired
    )

    plt.colorbar(image)

plot_logreg_2D(X, y, model)

**Вопрос:** Как на картинке визуализирована область, где модель не уверена ($p(y=+1|x) = 0.5$)? Как это обосновать теоритечески?



**Ответ:** # your answer here

#### __Задание 2. Калибровка вероятностей__ (1 балл)



Перейдём к оценке качества выдаваемых алгоритмами вероятностей. Начнём с калибровочных кривых.

Допустим, алгоритм возвращает некоторые числа от нуля до единицы. Хорошо ли они оценивают вероятность?

Хорошо откалиброванный  классификатор должен выдавать значения так, чтобы среди образцов, для которых он дал значение, близкое к $\alpha$, примерно $\alpha * 100 \%$ фактически принадлежали к положительному классу. (Например, если классификатор выдает 0.3 для некоторых, то 30% из них должны принадлежать классу 1)

Для построения калибровочной криовой используем следующий алгоритм:

Разобьем отрезок $[0, 1]$ на несколько маленьких отрезков одинаковой длины.

Рассмотрим $i$-й отрезок с границами $[a_i, b_i]$ и предсказания $p_1, p_2, \dots, p_k$, которые попали в него. Пусть им соответствуют истинные ответы $y_1, y_2, \dots, y_k$. Если алгоритм выдает корректные вероятности, то среди этих истинных ответов должно быть примерно $(a_i + b_i) / 2$ единиц. Иными словами, если нарисовать кривую, у которой по оси X отложены центры отрезков, а по оси Y — доли единичных ответов этих в отрезках, то она должна оказаться диагональной.

Ниже приведена функция, которая должна рисовать такие кривые. В ней допущено две ошибки — найдите и исправьте их.

In [ ]:
def plot_calibration_curve(y_test, preds):
    bin_middle_points = []
    bin_real_ratios = []
    n_bins = 10
    for i in range(n_bins):
        l = 1.0 / n_bins * i
        r = 1.0 / n_bins * (i + 1)
        bin_middle_points.append((l - r) / 2)
        bin_real_ratios.append(np.min(y_test[(preds >= l) & (preds < r)] == 1))
    plt.figure(figsize=(6,6))
    plt.plot(bin_middle_points, bin_real_ratios)
    plt.ylim([-0.05, 1.05])
    plt.grid()

Сгенерируйте синтетические данные аналогично использованным в самом первом задании. Постройте калибровочные кривые на тестовой части для логистической регрессии и метода опорных векторов (не забудьте перевести его предсказания в $[0;1]$).

Отрисуйте калибровочную кривую идеально откалиброванной модели (диагональ)

In [ ]:
# your code here

**Вопрос**: хорошо ли откалиброваны кривые для SVM, логистической регрессии? Подумайте, как это следует из вида кривой

**Ответ:** # your answer here

Из формальных способов в этом убедиться есть знакомый вам LogLoss, который напрямую оценивает вероятности,
$$\text{LogLoss} = -\frac{1}{N}\sum_{i} \sum_{k \in {0. 1}}\log p_k[y_i = k]$$
а так же BrierScore, который подсчитывает отклонение между получившейся вероятностью и реальным значением таргета.
$$\text{BrierScore} = \frac{1}{N}\sum_{i} (p_i - y_i)^2$$
Посмотрите на них тоже и сделайте вывод

In [ ]:
# your code here

Изучите распределение ответов классификаторов при помощи гистограмм

In [ ]:
import seaborn as sns

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

sns.histplot(y_svc_proba, bins=50, kde=True, ax=axes[0], color='blue', alpha=0.7)
axes[0].set_title('SVM Predicted Probabilities')
axes[0].set_xlabel('Probability')
axes[0].set_ylabel('Frequency')

sns.histplot(y_log_reg_proba, bins=50, kde=True, ax=axes[1], color='green', alpha=0.7)
axes[1].set_title('Logistic Regression Predicted Probabilities')
axes[1].set_xlabel('Probability')
axes[1].set_ylabel('Frequency')

sns.histplot(y_random, bins=50, kde=True, ax=axes[2], color='red', alpha=0.7)
axes[2].set_title('Random Classifier Predicted Probabilities')
axes[2].set_xlabel('Probability')
axes[2].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

**Вопрос:** Чем они различаются? Чем вы можете объяснить это?

**Ответ:** \n\nSVM показывает бимодальное распределение с пиками вблизи 0 и 1 - модель уверена в своих предсказаниях, разделяя классы чётко.\n\nLogistic Regression имеет более плавное унимодальное распределение, сосредоточенное вокруг 0.5 - модель менее уверена, выдаёт предсказания ближе к порогу.\n\nRandom Classifier показывает равномерное распределение в диапазоне [0, 1] - случайные предсказания без уверенности.

Воспользуйтесь `CalibratedClassifierCV` из `sklearn` для калибровки вероятностей метода опорных векторов на обучении и постройте с его помощью  предсказания для тестовой выборки.

In [ ]:
# your code here

**Вопрос:** Улучшились ли калибровочная кривая и качество калибровки?

**Ответ:** # your answer here

##### __Бонус: Авторское решение__ (0.5 балла)

Реализуйте свою функцию для калибровки вероятностей, используя любой из известных подходов. Кратко опишите ваш подход и продемонстрируйте результаты. Ключевые слова для вдохновения: `Platt`, `Isotonic`.

In [ ]:
# your code here

# Часть 2. Обработка категориальных переменных (4 балла + 1.5 бонус)

Как мы знаем, перекодировать категориальную переменную в список чисел (к примеру 1, 2, 3, ..., n) плохо, поскольку это бы задало на множестве ее значений некоторый порядок, не имеющий смысла.

В этой части мы рассмотрим два основных способа обработки категориальных значений:
- One-hot-кодирование
- Счётчики (CTR, mean-target кодирование, ...) — каждый категориальный признак заменяется на среднее значение целевой переменной по всем объектам, имеющим одинаковое значение в этом признаке.

Начнём с one-hot-кодирования. Допустим наш категориальный признак $f_j(x)$ принимает значения из множества $C=\{c_1, \dots, c_m\}$. Заменим его на $m$ бинарных признаков $b_1(x), \dots, b_m(x)$, каждый из которых является индикатором одного из возможных категориальных значений:
$$
b_i(x) = [f_j(x) = c_i]
$$

#### __Подготовка данных__

*(бесценный шаг)*

Разберем датасет [покупок велосипедов](https://www.kaggle.com/datasets/heeraldedhia/bike-buyers/): даны признаки покупателя, требуется предсказать, купит ли он/она велосипед



Замените пропуски в категориальных переменных на новую категорию (`'undefined'`)

Разделите признаки на 2 таблицы: категориальные и числовые признаки

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.metrics import roc_auc_score
from sklearn.metrics import RocCurveDisplay
from sklearn.svm import LinearSVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.model_selection import KFold

In [2]:
import kagglehub

path = kagglehub.dataset_download("heeraldedhia/bike-buyers") +  "/bike_buyers.csv"


In [3]:
df = pd.read_csv(path)

In [4]:
cat_cols = df.select_dtypes(include=['object']).columns
num_cols = df.select_dtypes(include=['float64', 'int64']).columns

In [5]:
target = df['Purchased Bike'].map({'Yes': 1, 'No': 0})


In [6]:
df_cat = df[cat_cols].copy()
df_num = df[num_cols].copy()

In [7]:
df_cat.drop(columns=['Purchased Bike'], inplace=True)

In [8]:
df_cat.fillna('undefined', inplace=True)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    df_cat,
    target,
    test_size=0.3,
    random_state=42
)


В начале поработаем только с категориальными признаками

#### __Задание 3. OrdinalEncoder__  (0.5 балла)

Закодируйте категориальные признаки с помощью `OrdinalEncoder`. Посчитайте качество (в этом задании будем работать c __`AUC-PR`__) при применении логистической регрессии. Замерьте время, потребовавшееся на обучение модели, с учетом кодирования признаков.

In [10]:
results = pd.DataFrame(columns=['Encoder', 'AUC_PR', 'Time'])

In [36]:
import time
from sklearn.metrics import average_precision_score
import random

SEED = 42
np.random.seed(SEED)
random.seed(SEED)

def eval_encoder(encoder, encoder_name, X_train_in, X_test_in, y_train_in=None):
    start = time.time()
    
    if y_train_in is None:
        X_train_enc = encoder.fit_transform(X_train_in)
    else:
        X_train_enc = encoder.fit_transform(X_train_in, y_train_in)
    X_test_enc = encoder.transform(X_test_in)
    
    log_reg = LogisticRegression(
        C=1.0,
        penalty="l2",
        tol=0.0001,
        random_state=42,
        max_iter=1000
    )

    params_log_reg = {
        'C': np.logspace(-9, 4, 20),
        'tol': np.logspace(-8, -2, 10)
    }
    kfold = KFold(5, random_state=42, shuffle=True)

    random_search_log_reg = RandomizedSearchCV(
        estimator=log_reg,
        param_distributions=params_log_reg,
        cv=kfold,
        n_iter=60,
        scoring='average_precision',
        n_jobs=-1,
        random_state=42,
        verbose=0  # Изменено с 1 на 0 для меньшего вывода
    )

    random_search_log_reg.fit(X_train_enc, y_train)
    
    # Используем best_estimator_ для предсказаний
    y_pred_proba = random_search_log_reg.best_estimator_.predict_proba(X_test_enc)[:, 1]
    
    auc_pr = average_precision_score(y_test, y_pred_proba)
    
    end = time.time()
    sum_time = round(end - start, 6)
    
    # Add results to Data Frame
    global results
    new_row = pd.DataFrame({
        'Encoder': [encoder_name], 
        'AUC_PR': [auc_pr], 
        'Time': [sum_time]
    })
    
    results = pd.concat([results, new_row], ignore_index=True)
    
    print(f"{encoder_name}: AUC-PR = {auc_pr:.4f}, Time = {sum_time:.4f}s")

    return auc_pr

In [12]:
from sklearn.preprocessing import OrdinalEncoder

ord_enc = OrdinalEncoder()

eval_encoder(ord_enc, 'OrdinalEncoder', X_train, X_test)

OrdinalEncoder: AUC-PR = 0.5886, Time = 2.3525s


/var/folders/8r/46ncjmbn5rq0s4d0hqmk4f440000gn/T/ipykernel_1523/3131977700.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, new_row], ignore_index=True)


np.float64(0.5886252772025165)

#### __Задание 4. One-Hot Encoding__ (0.5 балла)



Закодируйте все категориальные признаки с помощью one-hot-кодирования. Обучите логистическую регрессию и посмотрите, как изменилось качество модели (в сравнении с тем, что было до кодирования). Измерьте время, потребовавшееся на кодирование категориальных признаков и обучение модели.

In [13]:
from sklearn.preprocessing import OneHotEncoder

one_hot_enc = OneHotEncoder()

eval_encoder(one_hot_enc, 'OneHotEncoder', X_train, X_test)

OneHotEncoder: AUC-PR = 0.6140, Time = 0.2411s


np.float64(0.6140415638575276)

In [14]:
results

,Encoder,AUC_PR,Time
0,OrdinalEncoder,0.588625,2.352460
1,OneHotEncoder,0.614042,0.241091


Как можно заметить, one-hot-кодирование может сильно увеличивать количество признаков. Это сказывается на объеме необходимой памяти, особенно, если некоторый признак имеет большое количество значений.


#### __Задание 5. Mean-target Encoding__ (1 балл)

> Проблемы разрастания числа признаков можно избежать в другом способе кодирования категориальных признаков — mean-target encoding (для простоты будем называть это __счётчиками__). Сравним эффективность методов в рамках нашей маркетинговой задачи.

> Основная идея в том, что важны не сами категории, а значения целевой переменной, которые имеют объекты этой категории. Каждый категориальный признак мы заменим средним значением целевой переменной по всем объектам этой же категории:

$$
g_j(x, X) = \frac{\sum_{i=1}^{\ell} [f_j(x) = f_j(x_i)][y_i = +1]}{\sum_{i=1}^{\ell} [f_j(x) = f_j(x_i)]}
$$

Закодируйте категориальные переменные с помощью счётчиков (ровно так, как описано выше, без каких-либо хитростей). Обучите логистическую регрессию и посмотрите на качество модели на тестовом множестве.

Сравните время обучения с предыдущими экспериментами (с учетом кодирования признаков).

##### __Бонус: Эффективная реализация (1 балл)__

Здесь и далее реализуйте вычисление счетчиков с помощью трансформера (наследуйтесь от классов `BaseEstimator, TransformerMixin` из `sklearn.base`). Обратите внимание, что все вычисления должны быть векторизованными, трансформер не должен модифицировать передаваемую ему выборку inplace, а все необходимые статистики нужно считать только по обучающей выборке в методе `fit`. Ваш трансформер должен принимать при инициализации список из категориальных признаков и изменять только их.

In [52]:
np.random.normal(0, 1)

-0.5622875292409727

In [16]:
from sklearn.base import BaseEstimator, TransformerMixin

class MeanTargetEncoder(TransformerMixin, BaseEstimator):
    def __init__(self, sigma):
        self.global_mean = None
        self.encoding_dicts = {}
        self.sigma = sigma


    def fit(self, X, y):
        X = X.copy()
        X = pd.concat([X, y], axis=1)
        self.global_mean = y.mean()

        for col in X.columns:
            mean_dict = {}

            for category in X[col].unique():
                if pd.isna(category):
                    continue
                mean_dict[category] = X[X[col] == category]['Purchased Bike'].mean() + np.random.normal(0, self.sigma)
            
            self.encoding_dicts[col] = mean_dict
        return self
    
    def transform(self, X):
        X = X.copy()
        for col in X.columns:
            X[f'{col} Enc'] = X[col].map(self.encoding_dicts[col])
            X.drop(columns=[col], inplace=True)
            X[f'{col} Enc'] = X[f'{col} Enc'].fillna(self.global_mean)

        return X

In [55]:
mean_target_enc = MeanTargetEncoder(sigma = 0.055)
eval_encoder(mean_target_enc, 'MeanTargetEncoder', X_train, X_test, y_train)

MeanTargetEncoder: AUC-PR = 0.6007, Time = 0.2655s


np.float64(0.6007305185392742)

_______

__Методы борьбы с переобучением счетчиков__


Отметим, что mean-target encoding признаки сами по себе являются классификаторами и, обучаясь на них, мы допускаем "утечку" целевой переменной в признаки. Это ведёт к __переобучению__, поэтому считать такие признаки необходимо таким образом, чтобы при вычислении для конкретного объекта его __целевая метка не использовалась__.

Это можно делать следующими способами:
1. Вычислять значение счётчика по всем объектам расположенным выше в датасете (например, если у нас выборка отсортирована по времени).
2. Вычислять по фолдам, то есть делить выборку на некоторое количество частей и подсчитывать значение признаков по всем фолдам кроме текущего (как делается в кросс-валидации).
3. Внесение некоторого шума в посчитанные признаки.

#### __Задание 6. Пошумим__  (0.5 балла)

Реализуйте корректное вычисление счётчиков самым простым способом — добавление шума к значениям.  При этом постарайтесь найти баланс между борьбой с переобучением и сохранением полезности признаков. Снова обучите логистическую регрессию, оцените качество.

In [27]:
mean_target_enc = MeanTargetEncoder(0.05599)
auc_pr_value = eval_encoder(mean_target_enc, f'MeanTargetEncoder_0.094', X_train, X_test, y_train)


MeanTargetEncoder_0.094: AUC-PR = 0.6005, Time = 0.2721s


In [ ]:
# auc_pr_list = []
# for sigma in np.linspace(0.05, 0.12, 200):
#     mean_target_enc = MeanTargetEncoder(sigma)
#     auc_pr_value = eval_encoder(mean_target_enc, f'MeanTargetEncoder_{sigma:.5f}', X_train, X_test, y_train)
#     auc_pr_list.append(auc_pr_value)
# print(max(auc_pr_list))

**Вопрос:** Сделайте выводы. Помогло ли добавление шума? Почему?

**Ответ:** # your answer here

##### __Бонус: другой подход__ (0.5 балла)

Посчитайте корректные счётчики первым или вторым способов из описанных выше (не забудьте добавить и шум).




In [87]:
class MeanTargetEncoderCV(TransformerMixin, BaseEstimator):
    def __init__(self, sigma: float = 0, cv_num: int = 5, 
                 random_state: int = 42):

        self.global_mean = None
        self.cv_encodings = {}  # Энкодинги для каждого фолда
        self.train_encodings = {}  # Энкодинги на всем трейне для новых данных
        self.sigma = sigma
        self.cv_num = cv_num
        self.random_state = random_state
        self.columns_to_encode = []
        self.train_idx_by_fold = []  # Индексы трейна для каждого фолда
        
    def fit(self, X: pd.DataFrame, y: pd.Series):

        X = X.copy()
        y = y.copy()
        
        # Сохраняем глобальное среднее
        self.global_mean = y.mean()
        
        # Определяем колонки для кодирования
        self.len_train = X.shape[0]
        self.columns_to_encode = X.columns.tolist()
        
        # Инициализируем структуры
        self.cv_encodings = {fold: {} for fold in range(self.cv_num)}
        self.train_encodings = {}
        self.train_idx_by_fold = []
        
        # Создаем KFold
        cv = KFold(n_splits=self.cv_num, shuffle=True, random_state=self.random_state)
        
        # Сохраняем индексы для каждого фолда
        folds_indices = list(cv.split(X, y))
        
        # 1. Сохраняем энкодинги для каждого фолда (для трансформации трейна)
        for fold_idx, (train_idx, val_idx) in enumerate(folds_indices):
            self.train_idx_by_fold.append((train_idx, val_idx))
            
            X_train_fold = X.iloc[train_idx]
            y_train_fold = y.iloc[train_idx]
            
            for col in self.columns_to_encode:
                # Вычисляем энкодинг на трейне фолда
                temp_df = pd.DataFrame({
                    'feature': X_train_fold[col],
                    'target': y_train_fold
                })
                
                mean_by_category = temp_df.groupby('feature')['target'].mean()
                
                # Добавляем шум если sigma > 0
                if self.sigma > 0:
                    noise = np.random.normal(0, self.sigma, size=len(mean_by_category))
                    mean_by_category = mean_by_category + noise
                
                # Сохраняем для этого фолда
                self.cv_encodings[fold_idx][col] = mean_by_category.to_dict()
        
        # 2. Вычисляем энкодинги на всем трейне (для трансформации новых данных)
        for col in self.columns_to_encode:
            temp_df = pd.DataFrame({
                'feature': X[col],
                'target': y
            })
            
            mean_by_category = temp_df.groupby('feature')['target'].mean()
            
            if self.sigma > 0:
                noise = np.random.normal(0, self.sigma, size=len(mean_by_category))
                mean_by_category = mean_by_category + noise
            
            self.train_encodings[col] = mean_by_category.to_dict()
        
        return self
    
    def transform(self, X):
        X = X.copy()
        X = X.reset_index(drop=True)
        for col in self.columns_to_encode:
            X[f'{col} Enc'] = np.zeros((X.shape[0], 1))
        
        if X.shape[0] == self.len_train:

            for fold_idx, (train_idx, val_idx) in enumerate(self.train_idx_by_fold):
                for col in self.columns_to_encode:
                    X.loc[val_idx, f'{col} Enc'] = X.loc[val_idx, col].map(
                        self.cv_encodings[fold_idx][col]
                    )
            
            for col in self.columns_to_encode:
                X[f'{col} Enc'] = X[f'{col} Enc'].fillna(self.global_mean)
            
            X.drop(columns=self.columns_to_encode, inplace=True)

        return X

In [88]:
mean_target_enc_cv = MeanTargetEncoderCV()

mean_target_enc_cv.fit(X_train, y_train)



MeanTargetEncoderCV()

In [89]:
X_train_mean_enc_cv = mean_target_enc_cv.transform(X_train)

In [90]:
X_train_mean_enc_cv

,Marital Status Enc,Gender Enc,Education Enc,Occupation Enc,Home Owner Enc,Commute Distance Enc,Region Enc
0,0.426573,0.480287,0.550000,0.442857,0.474801,0.434343,0.438163
1,0.533582,0.472028,0.522581,0.529801,0.469274,0.233333,0.543689
2,0.505792,0.485401,0.553398,0.472727,0.430233,0.539604,0.460606
3,0.536965,0.500000,0.476510,0.412371,0.449438,0.212766,0.579832
4,0.407666,0.472028,0.490066,0.529801,0.473545,0.554455,0.424028
...,...,...,...,...,...,...,...
695,0.427609,0.485401,0.425676,0.421429,0.480519,0.372727,0.586538
696,0.505792,0.444840,0.425676,0.409091,0.480519,0.539604,0.460606
697,0.533582,0.472028,0.490066,0.430657,0.473545,0.388430,0.424028
698,0.536965,0.463235,0.562500,0.517857,0.449438,0.575000,0.500000


In [91]:
auc_pr_list = []
for sigma in np.linspace(0.005, 0.9, 20):
    mean_target_enc = MeanTargetEncoderCV(sigma)
    auc_pr_value = eval_encoder(mean_target_enc, f'MeanTargetEncoder_{sigma:.5f}', X_train, X_test, y_train)
    auc_pr_list.append(auc_pr_value)
print(max(auc_pr_list))

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Commute Distance
- Education
- Gender
- Home Owner
- Marital Status
- ...


#### __Задание 7. Сглаживание счетчиков__  (1 балл)

> Теперь ответим на следующий вопрос: что будет, если некоторая категория встречается в выборке всего несколько раз? По этой причине производится сглаживание счётчиков. Например, на практике хорошие результаты показывает использование сглаживания средним по всей выборке:
$$
g_j(x, X) = \frac{\sum_{i=1}^{\ell} [f_j(x) = f_j(x_i)][y_i = +1] + C \times \text{global_mean}}{\sum_{i=1}^{\ell} [f_j(x) = f_j(x_i)] + C}
$$
где $\text{global_mean}$ — доля объектов положительного класса в выборке, $C$ — параметр, определяющий степень сглаживания (можно использовать 10 или подобрать для каждого признака свой). Идея в том, что мы "разбавляем" среднее значение по категории глобальным средним значением. И тем меньше, чем большее количество объектов этой категории встречается в выборке.

> Вместо среднего значения целевой переменной для сглаживания можно использовать любое другое значение от 0 до 1 (этот параметр иногда называют $prior$). Можно сделать несколько признаков с разными значениями параметра. На практике в задачах бинарной классификации полезными бывают даже отрицательные значения!

Добавьте сглаживание, описанное выше и повторите эксперименты. Подберите $C$, чтобы качество было лучше, чем при использовании One-Hot-Encoding


In [ ]:
# your code here

#### **Задание 8. Числовые или категориальные?**  (0.5 балла)

Теперь добавим числовые признаки к счётчикам (тем, которые дали наибольший прирост качества).


Проверьте их на наличие выбросов и заполните пропуски средним или медианой, подумайте, что лучше в условиях наших данных



In [ ]:
# your code here

 Сейчас для числовых признаков мы ищем линейную зависимость, что в общем случае  может быть неверной гипотезой. Тем не менее, у этих признаков есть довольно много уникальных значений (сколько?), поэтому применять к ним one-hot кодирование может оказаться излишним. Попробуйте закодировать эти признаки с помощью счетчиков. Стало ли лучше?

In [ ]:
# your code here

> __Замечание.__ Усложнение методов вычисления счётчиков не делают результаты модели гарантированно лучше. Особенно с учётом того, что логистическая регрессия не такая сложная модель, чтобы переобучаться. Поэтому вы необязательно должны были получать на каждом шаге всё лучшие и лучшие результаты (но необходимые результаты у вас должны были получиться).



Как мы могли пронаблюдать, счётчики являются конкурентной альтернативой one-hot-кодированию. Опишите, какие плюсы и минусы использования счётчиков по сравнению с one-hot-кодированием вы заметили.

__Ответ:__ # your answer here

# Часть 3. Отбор признаков (2 балла)

Загрузим данные [UCI Adult Dataset](https://archive.ics.uci.edu/ml/datasets/Adult). Этот набор данных содержит информацию о годовых доходах отдельных людей. В качестве признакового описания используется различная информация о человеке (образование, профессия, брачный статус и т.д.). Целевая переменная является бинарной: больше ли годовой доход 50K долларов или нет.

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data

In [ ]:
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num',
    'marital-status', 'occupation', 'relationship', 'race', 'sex',
    'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
    'income'
]

df = pd.read_csv('adult.data', header=None, names=columns)
df['income'] = (df['income'] != " <=50K").astype('int32')
df.sample()

Важной частью процесса построения модели является отбор признаков. На практике многие признаки оказывают малое влияние на модель (при этом их расчёт занимает время) или даже негативно сказываются на качестве модели. Попробуем несколько подходов отбора признаков, оценим, как они влияют на качество модели и сколько времени занимают.



Разделите выборку на обучающую и тестовую в соотношении 3:1. Зафиксируйте `random_state=777`, также используйте `stratify=True`.

In [ ]:
# your code here

Давайте закодируем все категориальные признаки с помощью One-hot Encoding. Сколько новых признаков мы получим?

In [ ]:
# your code here

В качестве основной модели будем использовать логистическую регрессию, а целевой метрики — `AUC-PR`. Обучите модель и посчитайте качество на тестовой выборке. Давайте запомним полученное значение.

In [ ]:
# your code here

Допустим, мы хотим оставить только 40 лучших признаков.

Заметим, что нельзя оценивать качество по тестовой выборке, иначе мы можем переобучиться, как, например, при настройке гиперпараметров. Разделите обучающую выборку на 2 части, одну из которых, используйте для валидации. Исходную тестовую выборку стоит использовать только для финальной оценки качества после процедуры фильтрации

In [ ]:
# your code here

Попробуем сделать это следующими способами:

#### __Задание 9. Встроенные методы (0.5 балла)__

Начнём с отбора признаков с помощью модели. У разных алгоритмов есть разные встроенные способы оценки вклада признаков в предсказание. Как известно, у линейной модели за это отвечают веса, а значит, их модуль можно интерпретировать как важность. Такой метод отбора называются встроенным или embedded method, так как он заложен в особенности модели.

Оставьте 40 признаков с наибольшим модулем соответствующего параметра линейной модели. Обучите модели заново и оцените её качество. Замерьте скорость такого отбора признаков.



In [ ]:
# your code here

Изменилось ли качество? Как?

Подумаем, что мы не учли. Мы действовали в предположении, что признаки вносят вклад равномерно, и не учитывали их масштаб. Если мы умножим один из признаков в 100 раз, то без учёта регуляризации его вес уменьшится в эти же 100 раз. А мы на основе этого отбираем признаки! Давайте сначала отмасштабируем признаки одним из способов, а только потом будем удалять признаки.

Помните, что не все способы одинаково хороши, особенно в условиях наличия выбросов

Кстати, в таком случае надо пересчитать качество на всех признаках (сделайте это ниже). Если вы сделали нормирование признаков в самом начале, то попробуйте отобрать признаки на неотмасштабированных данных.

Что получилось?

In [ ]:
# your code here

Вопрос на засыпку: one-hot кодирование возвращает нам единичные признаки-индикаторы. Попробуйте также отскалировать их, как и обычные числовые, и снова выбрать 40 главных по вкладу признаков. Изменился ли их список? Изменится ли качество?

In [ ]:
# your code here

#### __Задание 10. Методы фильтрации (0.5 балла)__


Давайте отбирать признаки умнее, а именно через подсчёт некоторой функции для каждого признака. На основании значений этой функции будем оставлять наиболее важные признаки. Методы этого семейства называют фильтрующими или  filter methods.

Одна из самых простых функция - корреляция между признаком и целевой переменной. Подумайте, какая взаимосвязь между корреляцией и предсказательной способностью модели, и как бы вы использовали информацию о корреляции для отбора признаков

**Ответ:** # your code here

Посчитайте корреляцию каждого признака с таргетом и отфильтруйте 40 признаков исходя из того, что вы описали, после чего замерьте качество и время отбора



In [ ]:
# your code here

В качестве еще одной функция можно считать t-статистику:

$$t(j) = \frac{|\mu_+ - \mu_-|}{\sqrt{\frac{n_+ \sigma^2_+ + n_- \sigma^2_-}{n_+ + n_-}}},$$

где $\mu$, $\sigma$, $n$ соответственно среднее, стандартное отклонение и количество объектов каждого из классов.

Оставьте 40 признаков с наибольшим значением $t$, замерьте качество и скорость отбора признаков.

In [ ]:
# your code here

#### __Задание 11. Методы-обёртки__ (1 балл)

Третий из рассматриваемых нами методов работает следующим образом: мы исключаем признаки по очереди и смотрим, как это влияет на качество. Удаляем признаки таким жадным способом, пока не окажется выполненым некоторое условие (количество признаков или ухудшение качества). Более конкретно, алгоритм выглядит так:

- $k$ - число признаков, которых мы хотим оставить
- $m$ - число признаков, которых мы выбрасываем на каждой итерации, оно же длина шага

Шаг $i$:
- $F_i$ - набор признаков (равный всему множеству признаков на i=0)
- $M_i$ - их число, в общем случае $\max(k, M_{i-1} - m)$
1. Если признаков осталось ровно $k$, либо метрика стала уменьшаться более, чем на $\epsilon$ — останавливаемся (не наш случай, но так тоже можно)
2. Обучаем модель $a_i$ на наборе $F_i$, после чего оцениваем важность признаков (любым из способов выше или какими-нибудь ещё)
3. Отбираем $\min(M_i - k, m)$ наиболее бесполезных, согласно пункту 2, признаков (берем $m$, если можем, иначе оставляем вплоть до k), удаляем, переходим к следующему шагу

Снова оставьте только 40 признаков и оцените качество на тестовой выборке. Подберите длину шага из каких-то соображений (каких, кстати?) и замерьте время работы метода

In [ ]:
# your code here

Стоит отметить, что с помощью такого метода можно пойти и в обратную сторону. Попробуйте _добавлять_ самые полезные признаки в выборку до тех пор, пока не наберется 40 штук. Найдется ли порог, при котором добавление следующих признаков будет только ухудшать качество модели?

In [ ]:
# your code here

Давайте подведём итоги по отбору признаков. Назовите преимущества и недостатки каждого из методов. Какой метод привёл к наилучшему качеству?

**Ответ:** # your code here

# Часть 4. Оценка экономического эффекта модели (2 балла)



В данной части мы займемся тем, что от вас скорее всего потребуется на реальной работе (помимо перекладки `json`, разумеется). А именно:
- мы соберем несколько специализированных метрик качества,
- попытаемся настроить модель на максимизацию _прибыли_,
- оценим, сколько вообще получится заработать на этом.

Разумеется, здесь будет сделано множество упрощающих жизнь допущений, но обо всем по порядку. Если вы всё прослушали на экономике, то напомним, что выручка — это сколько денег нам принесли клиенты, а прибыль — выручка за вычетом расходов на зарплату и прочее.


#### __Задание 12. Прогноз по доходам и расходам__ (1 балл)

В этой части мы будем работать с данными [UCI Bank Marketing Dataset](https://archive.ics.uci.edu/ml/datasets/bank+marketing). Этот датасет содержит информацию о банковском телефонном маркетинге.

__Объектом__ здесь является телефонный звонок потенциальному клиенту с предложением некоторой услуги (утверждается, что это краткосрочный депозит). В качестве признакового описания используются характеристики клиента (образование, брак и т.д.), данные о звонке и различные экономические индикаторы - более подробная информация представлена в файле `bank-additional-names.txt`.
__Целевая переменная__ - ответ клиента (согласился ли он открыть депозит?)

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank-additional.zip
!unzip bank-additional.zip
df = pd.read_csv('bank-additional/bank-additional-full.csv', sep=';')

In [ ]:
df.head()

In [ ]:
X = df.drop(columns=['duration', 'y'])
y = (df.y == 'yes')

В этой части не нужно делить выборку - мы будем использовать кросс-валидацию.  Используйте наиболее подходящие с вашей точки зрения параметры и их значения (`shuffle`, `stratify`, число фолдов, ...). По кросс-валидации у вас получится несколько вариантов обучающей и тестовой выборки. Для удобства можно воспользоваться шаблоном ниже, который по ходу выполнения задания будет обрастать функционалом. Как обычно, это необязательно, но сохранять результаты экспериментов очень и очень желательно, в конце мы будем их сравнивать

In [ ]:
from collections import defaultdict
from sklearn.model_selection import KFold

def cross_validate(
    X,
    y,
    n_splits=5,
    random_state=None,
    shuffle=False,
    # другие аргументы, которые могут вам пригодиться дальше по пунктам
):
    metrics = []
    # или любой другой фолд, посмотрите в model_selection
    kf = KFold(n_splits=n_splits, random_state=random_state, shuffle=shuffle)

    for train_index, test_index in kf.split(X):

        # возьмите датасет и обучите модель
        # your code here

        # посчитайте метрики, которые вам нужны и добавьте результаты с каждого фолда
        metric_dict = {
            # "metric_key": metric_value
        }
        metrics.append(metric_dict)

    # осталось только красиво всё обернуть
    return pd.DataFrame(metrics)

Выберите метрику классификации, которая вам кажется подходящей, и обучите логистическую регрессию на каждой обучающей выборке (закодируйте категориальные признаки способом, который выше вам понравился больше всех, отнормируйте числовые, гиперпараметры оставьте по умолчанию), сделайте предсказания для соответствующих тестовых выборок, выведите результаты

In [ ]:
# your code here

Допустим, работники вашего колл-центра получают за один звонок клиенту 2 доллара. При согласии клиента на предлагаемые условия он принесет в банк 10 долларов. Предположим, что всем положительным прогнозам ваши сотрудники решили позвонить.

В качестве бизнес-метрики в нашей задаче мы будем считать прибыль aka `profit`, соответственно лучшую модель будем выбирать исходя из этого.
Посчитайте на всех тестовых выборках выручку и сохраните результаты для бизнес-метрики вместе с предыдущей метрикой, которую вы выбрали

Ответьте на вопросы:
- Сколько денег вы в среднем заработаете?
- Какое получилось стандартное отклонение профита?
- Сколько из заработанных денег придётся отдать операторам вашего колл-центра?
- Пропорциональна ли бизнес-метрика выбранной метрике классификации?

In [ ]:
# your code here

Внесем некоторую долю случайности. Пусть теперь согласный на условия клиент будет приносить не 10 долларов, а случайную величину, равномерно распределенную в интервале $[0;20)$. Проделайте все те же самые действия. Для имитации реальной ситуации **НЕ** фиксируйте `random_seed` при подсчете выручки с клиента (для разбиения на фолды разумеется оставьте). Что получилось?

In [ ]:
# your code here

Настройте по кросс-валидации коэффициент регуляризации модели для максимизации прибыли (считайте как случайную величину выше). Удалось ли получить какой-то выигрыш? При каком коэффициенте регуляризациии прибыль максимальна? Постройте график зависимости ожидаемой прибыли от коэффициента

In [ ]:
# your code here

Попробуйте запустить перебор несколько раз. Находится ли каждый раз один и тот же "лучший" коэффициент? Присутствует ли какая-то закономерность? Какие вы можете сделать из этого выводы?

In [ ]:
# your code here

#### __Задание 13. Ключевая метрика__ (1 балл)

Выше мы уже описали примерную экономическую модель вашей задачи. Как вы считаете, что для вашего бизнеса важнее — хороший precision или recall модели? Почему?

__Ответ:__ # your answer here



> Вспомним, что на самом деле логистическая регрессия предсказывает нам вероятности положительного класса для объекта. Возможно, путем настройки __порога бинаризации__ этих вероятностей мы сможем получить какой-то выигрыш?

Проверьте ваши рассуждения выше с помощью настройки порога бинаризации на кросс-валидации для максимизации прибыли. Воспользуйтесь сеткой от 0 до 1 с шагом 0.01. Напомним, что снижение порога дает нам более высокий recall и более низкий precision, и наоборот. Добавьте новую ML-метрику в ваш CV-пайплайн, найдите такой порог, при котором бизнес-метрика максимальна, и проверьте, связана ли новая ML метрика с профитом

In [ ]:
# your code here

Постройте график зависимости прибыли от порога бинаризации. Выделите наилучший порог




In [ ]:
# your code here

__Вопрос:__ Замечаете ли вы какую-то закономерность? Для правильного ответа на этот вопрос попробуйте запустить несколько раз и задумайтесь, почему порог получается в какой-то конкретной области?

__Ответ:__ # your answer here

Наконец, чтобы точнее понять, что наша модель лучше исходной, посчитайте среднее и стандартное отклонение по фолдам бизнес-метрики для оптимизированной модели (гиперпараметры + порог) и дефолтной логистической регрессии. Проверьте, действительно ли удалось добиться значимого изменения прибыли — примените какой-либо статистический тест (например, парный t-критерий с $\alpha=0.95$) к метрике, полученной двумя этими моделями

In [ ]:
# your code here

# __Бонусная часть. Многоклассовая классификация__ (1.5 балла)

Как известно, некоторые задачи не ограничиваются всего лишь двумя классами. На лекции вы проходили несколько способов обобщения линейных моделей на этот случай: One-vs-Rest и One-vs-One. Ниже мы посмотрим, в чём преимущества и недостатки обоих подходов, а так же попробуем ещё один чуть более экзотический метод

#### **Задание 14. One-vs-Rest vs One-vs-One** (0.5 балла)

В качестве [датасета](https://www.kaggle.com/datasets/thedevastator/higher-education-predictors-of-student-retention/data) здесь и ниже мы будем брать очень жизненные и актуальные данные о том, доучится студент или нет, в зависимости от курсов, возраста, гендера и прочих (не)осуждаемых признаков.

In [ ]:
import kagglehub

path = kagglehub.dataset_download("thedevastator/higher-education-predictors-of-student-retention") + "/dataset.csv"

features = ["Marital status", "Course", "Nacionality", "Gender", "Age at enrollment"]
target = "Target"

Будем смотреть только какое-то подмножество наиболее весёлых факторов. От вас по классике потребуется их преобразовать, в зависимости от того, числовые они или категориальные и **закодировать таргет чиселками!!!**

In [ ]:
# your code here

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=228, shuffle=True, test_size=0.2)

Ваш следующий шаг - посмотреть, каким образом в `sklearn` реализованы OvR и OvO, обучить таким образом логистическую регрессию с `max_iter=10000`, далее выбрать какую-то метрику (и её усреднение, его выбор тоже аргументируйте), и сравнить следующие параметры:
- число классификаторов
- скорость обучения
- качество модели

Также сохраните куда-нибудь предсказания вероятностей у каждой из моделей. Это можно сделать не одним способом, но возможно вам чуть с этим поможет следующий пункт


In [ ]:
# your code here

Как вы объясните полученные результаты?

__Ответ:__ # your code here

#### __Задание 15. Softmax регрессия__ (1 балл)

Однако любознательные машинисты могут задаться вопросом "А зачем нам вся эта шляпа, если у сигмоиды есть обобщение на случай многоклассовой классификации?" Если вам понравилось считать градиенты в прошлом дз, или вам нравится обучать нейросети, этот пункт для вас. Здесь мы попробуем построить одну-единственную модель, которая будет всё предсказывать, а также сравним с вариантами выше

Начнём с подсчёта лосса. Вспомним, что логистическая функция потерь это частный случай кросс-энтропии, её и будем пытаться оптимизировать.

$$
\text{CE}(X, y) = -\frac{1}{N}\sum_i \sum_k [y_i = k] \log p(x_i = k)
$$
Вероятности в данном случае будем считать при помощи софтмакса, что есть общий случай сигмоиды

$$
p(x_i) = \text{Softmax}(a(x_i)); \quad
\text{Softmax}(x)_k = \frac{e^{x_{k}}}{\sum_j e^{x_{j}}} \\
$$

Предсказание модели на одном объекта будет делаться уже при помощи матрицы весов, посклоьку выходов несколько

$$
a(x_i) = x_i\cdot W \\
$$

Ниже предлагается написать код для такой функции потерь. Если необходимо, модифицируйте шаблон по своему усмотрению (вспомогательные функции, новые аргументы, всё, что душа пожелает)

In [ ]:
from typing import Iterable, Optional
from torch.nn.functional import cross_entropy
import torch

def custom_ce(
    y_pred: np.ndarray[float],
    y_true: np.ndarray[int],
) -> float:
    # your code here
    return

In [ ]:
for _ in range(1000):

    n_objects = np.random.randint(1, 100)
    n_classes = np.random.randint(2, 20)
    y_pred = np.random.normal(0, 1, (n_objects, n_classes))
    y_true = np.random.randint(low=0, high=n_classes, size=(n_objects,))

    your_ce = custom_ce(y_pred, y_true) # не забудьте поправить, если меняли шаблон
    torch_ce = cross_entropy(torch.tensor(y_pred), torch.tensor(y_true))
    assert np.allclose(your_ce, torch_ce), "Что-то пошло не так"

Дальше самая интересная часть - нужно вывести производную этой функции потерь (на всякий случай уточним, что `torch` использовать нельзя, разве что для самопроверки). Полезные факты, которые вам могут пригодиться:

- в матричном виде найти производную непросто, попробуйте сперва сделать это для одного объекта, обобщить будет полегче
- логсофтмакс дифференцировать гораздо легче, чем просто софтмакс
- не забывайте про правило дифференцирования сложной функции
- поскольку веса в данном случае матрица, результат будет тоже матрица, учтите при сверке размерностей
- если вы не придумали, как преобразовать индикаторы в векторный вид, сейчас самое время

In [ ]:
def ce_gradient(X: np.ndarray, W: np.ndarray, y: np.ndarray) -> np.ndarray[float]:
    # your code here
    return

Дальше дело за малым. Вспомните (или узнайте), как делается градиентный спуск, и дополните класс софтмакс-регрессии ниже. Здесь разумнее использовать критерий останова по итерациям, но логрег из `sklearn` устроен немного хитрее. Если хотите добавить еще критерии останова, какие-то другие параметры, то пожалуйста

In [ ]:
class SoftmaxRegression:

    def __init__(self, lr=1e-3, max_iter=10000):
        self.W = None
        self.max_iter = max_iter
        self.lr = lr

    def fit(self, X, y):
        # your code here

    def predict(self, X, y=None):
        # your code here

Обучите на тех же данных, что и выше, замерьте те же три параметра, плюс сравните значения кросс-энтропии для уже трёх моделей. Сравните модели между собой и выберите фаворита в данной задаче.

In [ ]:
# your code here

__Ответ__: # your text here

__Бонус (0.01 балла):__ что вы кушали в день сдачи данного ДЗ на завтрак?

__Ответ:__ # your answer here